# Atividade 3

Faça tudo via dbt. 

Envia como resposta:
- A query SQL
- Comando utilizado via terminal no `dbt`
- Um print do resultado no próprio notebook, para isso é necessário criar uma pasta no repositório onde serão armazenadas as imagens, e para inserir no notebook é necessário criar uma célula `markdown` e inserir o seguinte comando: 

```markdown
![Nome da Foto](path/para/imagem)
```

1. Crie uma tabela `orders_by_year` a partir de `orders` que agrupa a quantidade de pedidos por ano, considerando a coluna `order_approved_at`

    Comando terminal

`` dbt run --models shop.orders_by_year ``



``` sql
{{ config(materialized='table') }}

   SELECT count(*) as qtde_pedidos,
       EXTRACT(YEAR FROM order_approved_at) as ano
FROM `dbt-shop-anderson.dbt_shop.orders` 
group by EXTRACT(YEAR FROM order_approved_at) 

```

![Alt text](imagens/questao1.png)

2. Crie uma `view` que agrega os totais de pedidos por estado. Considere as tabelas _customers_ para obter os estados (coluna _customer_state_) e _orders_ para contabilizar a quantidade de pedidos.

    Comando terminal

`` dbt run --models shop.orders_by_state ``

``` sql
SELECT count(b.order_id) as qtde_pedidos,
       customer_state
FROM `dbt-shop-anderson.dbt_shop.customers` a inner join 
`dbt-shop-anderson.dbt_shop.orders` b on a.customer_id = b.customer_id
group by customer_state 

```

![Q2](imagens/questao2.png)

3. De acordo com o resultado do exercício 2 retorne apenas os estados que tem mais de 5000 pedidos, esse número deve ser passado via variável `number_of_orders` do `dbt`.

    Comando terminal

`` dbt run --models shop.orders_by_state_5000 --vars "{'qtde_pedidos':'5000'}" ``

``` sql
select * from `dataset_anderson.orders_by_state` 
where qtde_pedidos > CAST('{{ var("qtde_pedidos") }}' as INT64)

```

![Q3](imagens/questao3.png)

4. Crie uma `view` chamada `customers_by_region` que utiliza os dados da tabela `customers`. A visão deve realizar uma agregação por região do Brasil (Norte, Nordeste, Sudeste, Centro Oeste e Sul) e contar o número de clientes em cada uma delas.

    Comando terminal

`` dbt run --models shop.customers_by_region ``

``` sql
SELECT 
  CASE 
    WHEN customer_state IN ('AM', 'AC', 'AP', 'PA', 'RO', 'RR', 'TO') THEN 'Norte'
    WHEN customer_state IN ('AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE') THEN 'Nordeste'
    WHEN customer_state IN ('ES', 'MG', 'RJ', 'SP') THEN 'Sudeste'
    WHEN customer_state IN ('DF', 'GO', 'MT', 'MS') THEN 'Centro-Oeste'
    WHEN customer_state IN ('PR', 'RS', 'SC') THEN 'Sul'
    ELSE 'Outro' 
  END AS region,
  COUNT(distinct(customer_unique_id)) AS num_customers
FROM `dbt-shop-anderson.dbt_shop.customers` 
GROUP BY region

```

![Q4](imagens/questao4.png)

5. Crie uma tabela `items_costs` que contenha o preço total de um item (soma do valor do produto com o frete).

    Comando terminal

`` dbt run --models shop.items_costs ``

``` sql
 {{ config(materialized='table') }}

SELECT  
*,
(price + freight_value) as preco_total

FROM `dbt-shop-anderson.dbt_shop.items` 

```

![Q5](imagens/questao5.png)

6. Crie uma `view` com o nome `orders_shipped_costs`, você deve calcular o preço total do pedido somado com o frete, apenas para pedidos enviado (shipped).

    Comando terminal

`` dbt run --models shop.orders_shipped_costs ``

``` sql
 {{ config(materialized='table') }}

SELECT 
orders.order_id,
orders.customer_id,
orders.order_status,
sum(price) + sum(freight_value) as preco_total
FROM `dbt-shop-anderson.dbt_shop.orders` orders 
inner join `dbt-shop-anderson.dbt_shop.items` items
on items.order_id = orders.order_id 
--and orders.order_status = 'shipped' 
--and orders.order_id = '053245dc8f6f668c40923c3aedcd384e'
group by orders.order_id,
orders.customer_id,
orders.order_status

-- Fernando não identifiquei a necessidade de fazer uma sub query, pois analisando as tabelas entendi que o frete esta atrelado ao item do pedido, já o status de enviado ou não está no nível do pedido e não do item como o frete, podemos falar na última aula sobre meu entendimento e corrigir se for o caso, ok?

```

![Q6](imagens/questao6.png)

In [1]:
!dbt --version

Core:
  - installed: 1.4.6
  - latest:    1.4.6 - Up to date!

Plugins:
  - bigquery: 1.4.3 - Up to date!
